In [ ]:
!git clone https://github.com/SKTBrain/KoBERT.git

In [ ]:
cd KoBERT

In [ ]:
!pip install tqdm # progressbar 사용

!pip install -r requirements.txt
!pip install .
!pip install torch

# transformers 의 경우 최신 버전으로 설치 시 에러 발생
!pip install transformers #==3

In [ ]:
import os
import pandas as pd
import numpy as np
import csv 
import sys

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
import gluonnlp as nlp
from torch.utils.data import Dataset, DataLoader

# from kobert_transformers import get_kobert_model, get_distilkobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook

from tqdm import trange

# 환경 세팅

In [ ]:
# pytorch 에러 때문에 버전 확인
print(sys.version)
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

In [ ]:
#GPU 사용을 위해
device = torch.device("cuda:0")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 학습된 모델 불러오기

In [ ]:
model_saved_path = "/content/drive/MyDrive/forColab/learned_model/model3.pt"

In [ ]:
# Setting parameters
max_len = 128 
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
class BERTDataset(Dataset):
    # sentences 의 index와 label index 매개변수 제거
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

      # sentences 리스트와 label 리스트를 만드는 코드 변경
        self.sentences = [transform([i]) for i in dataset['contents']]
        self.labels = list(dataset['label'])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768, # Dimensionality of the encoder layers and the pooler layer
                 num_classes = 3, # 분류할 class label의 개수                
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#광고 여부를 확인하는, 이미 학습된 모델 로딩
model_checkAd = torch.load(model_saved_path)

In [ ]:
# 이미 pre train 된 bert 모델과 vocab을 불러옴
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
# 위에서 불러온 vocab을 이용하여 토크나이저 선언
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 광고 유무 확인 코드

In [ ]:
class Advertisement:

  def check_ad(self, lst):
    _data = self.format_for_test(lst)
    _dataset = BERTDataset(_data, tok, max_len, True, False)
    _dataloader = torch.utils.data.DataLoader(_dataset, batch_size=batch_size, num_workers=5, shuffle=False)
    pred = np.mean(self.predict(_dataloader))
    if pred < 1: return True
    else: return False

  def format_for_test(self, sentence):
    data_format = pd.DataFrame()
    data_format['contents'] = sentence
    data_format['label'] = pd.Series([0] * len(sentence))
    return data_format

  def predict(self, _dataloader):
    lst_results = []
    model_checkAd.eval()
    # for batch_id, (token_ids, valid_length, segment_ids) in enumerate(_dataloader):
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model_checkAd(token_ids, valid_length, segment_ids)
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            lst_results.append(np.argmax(logits))
    return lst_results

# 광고 유무 판단

In [ ]:
data_name = 'gangnam_pizza'

dir_path = '/content/drive/MyDrive/forColab/final/' + data_name + '/'

file_save_path = dir_path + 'summary/'
os.mkdir(file_save_path)
save_file_path = file_save_path + data_name + '.csv'

In [ ]:
file_list = os.listdir(dir_path)

In [ ]:
ad = Advertisement()

In [ ]:
df_data = pd.DataFrame(columns=['title', 'restaurant', 'ad'])

for i in trange(len(file_list)):
  f = open(dir_path + 'blog_data_'+ str(i)+'.txt', 'r')
  file_text = f.readlines()
  f.close()

  # title
  title = file_text[2][1:-1]
  # restaurant
  restaurant = file_text[4][1:-1]
  # text
  text = [file_text[i][1:-1]for i in range(6, len(file_text)-2)]
  #prediction
  pred = ad.check_ad(text)

  df_data = df_data.append({'title': title, 'restaurant': restaurant, 'ad' : pred}, ignore_index=True)
df_data.to_csv(save_file_path, encoding='utf-8-sig', header=False, index=False)